# 一、 有关数据集
## 1. 数据集来源
平台：Kaggle（全球知名的数据科学竞赛平台）

项目名称：Titanic: Machine Learning from Disaster

数据集链接：[Titanic Dataset on Kaggle](https://www.kaggle.com/c/titanic/data)

## 2. 数据集内容
数据集包含两个主要文件：

+ train.csv（训练集）：891 条记录，包含乘客的基本信息和是否幸存（Survived 列）。

+ test.csv（测试集）：418 条记录，仅包含乘客的基本信息，用于预测是否幸存。

**字段说明**

|字段名|说明|
|  :----  | :----  |
|PassengerId|乘客ID（唯一标识）|
Survived|是否幸存（1=是，0=否）|
Pclass|舱位等级（1=头等舱，2=二等舱，3=三等舱）|
Name|乘客姓名|
Sex|性别（male=男性，female=女性）|
Age|年龄|
SibSp|同船兄弟姐妹/配偶数量|
Parch|同船父母/子女数量|
Ticket|船票编号|
Fare|船票价格|
Cabin|客舱号（部分缺失）|
Embarked|登船港口（C=Cherbourg, Q=Queenstown, S=Southampton）|

## 3. 使用场景
探索乘客特征与幸存率的关系（如性别、舱位等级、年龄等）。
# 二、 数据处理
## 1. 加载数据

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
# 加载数据集
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
# 查看数据基本信息
print(train_data.info())
print(test_data.info())


## 2. 数据概览
+ 训练集：891 条记录，12 列。

+ 测试集：418 条记录，11 列（缺少 Survived 列）。

## 3. 数据清洗
### （1）处理缺失值

In [ ]:
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())  # 年龄用中位数填充
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])  # 登船港口用众数填充
train_data = train_data.drop(columns=['Cabin'])  # 删除 Cabin 列（缺失率过高）

# 测试集同样处理
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())
test_data = test_data.drop(columns=['Cabin'])

### （2） 数据格式转换
+ 将 Sex 转换为数值型（male=0, female=1）。

+ 将 Embarked 转换为数值型（S=0, C=1, Q=2）。

In [ ]:
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
train_data['Embarked'] = train_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# 测试集同样处理
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})
test_data['Embarked'] = test_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# 三、 探索性分析

## 1. 幸存率分析

In [ ]:
# 总体幸存率
survival_rate = train_data['Survived'].mean()
print(f"Overall survival rate: {survival_rate:.2%}")

# 按性别分析
sns.barplot(x='Sex', y='Survived', data=train_data)
plt.title('Survival Rate by Sex')
plt.savefig('survival_by_sex.png')  # 保存图表
plt.close()
# 按舱位等级分析
sns.barplot(x='Pclass', y='Survived', data=train_data)
plt.title('Survival Rate by Pclass')
plt.savefig('survival_by_pclass.png')  # 保存图表
plt.close()

![按性别分析](survival_by_sex.png)

![按舱位等级分析](survival_by_pclass.png)
## 2. 年龄分布与幸存率

In [ ]:
# 年龄分布
sns.histplot(train_data['Age'], bins=20, kde=True)
plt.title('Age Distribution')
plt.savefig('age_distribution.png')  # 保存图表
plt.close()

# 年龄与幸存率
sns.boxplot(x='Survived', y='Age', data=train_data)
plt.title('Age Distribution by Survival')
plt.savefig('age_by_survival.png')  # 保存图表
plt.close()

![年龄分布](age_distribution.png)

![按年龄分析](age_by_survival.png)